## DEPENDÈNCIES

In [1]:
import pandas as pd
import folium
from scipy.spatial import Voronoi
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## CÀRREGA DE DF

In [3]:
df_lloguer = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'Lloguer_BCN_districte')

In [4]:
df_inflacio = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'inflacio')

In [5]:
df_lloguer_barri = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'Lloguer_BCN_barri')

In [6]:
df_compra = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'Compra_BCN_districte')

In [7]:
df_compra_barri = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'Compra_BCN_barri')

In [8]:
df_habitatges_turistic = pd.read_csv('habitatges_us_turistic.csv', delimiter=',')

df_habitatges_turistic = df_habitatges_turistic.dropna(subset=['LATITUD_Y', 'LONGITUD_X'])

df_habitatges_turistic = df_habitatges_turistic[
    (df_habitatges_turistic['LATITUD_Y'] != '') & 
    (df_habitatges_turistic['LONGITUD_X'] != '') & 
    (df_habitatges_turistic['LATITUD_Y'].apply(lambda x: isinstance(x, (int, float)))) &
    (df_habitatges_turistic['LONGITUD_X'].apply(lambda x: isinstance(x, (int, float))))
]


In [9]:
df_habitatges_turistic.head(2)

,N_EXPEDIENT,CODI_DISTRICTE,DISTRICTE,CODI_BARRI,BARRI,TIPUS_CARRER,CARRER,TIPUS_NUM,NUM1,LLETRA1,NUM2,LLETRA2,BLOC,PORTAL,ESCALA,PIS,PORTA,NUMERO_REGISTRE_GENERALITAT,NUMERO_PLACES,LONGITUD_X,LATITUD_Y
0,01-2011-0123,1.0,CIUTAT VELLA,4.0,"Sant Pere, Santa Caterina i la Ribera",Carrer,AGULLERS,2.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,HUTB-003654,3.0,2.180903,41.381039
1,01-2011-0137,1.0,CIUTAT VELLA,2.0,el Barri Gòtic,Carrer,ROCA,2.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2,HUTB-001678,3.0,2.171985,41.380379


In [10]:
df_renda_lloguer = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'renda_lloguer_BCN')

In [11]:
df_padro_obra_nova = pd.read_excel('conjunt_dades.xlsx', sheet_name = 'padro_obra_nova')
df_padro_obra_nova_districtes = df_padro_obra_nova.groupby(['any', 'districtes municipals']).sum().reset_index()

## CODI

### MAPA

#### DISTRICTES

In [12]:
# CÀLCUL DELS QUANTILS

df_quantils = df_lloguer.groupby('any')['lloguer'].quantile([0.25, 0.5, 0.75]).unstack()
df_quantils.columns = ['Q1', 'mediana/Q2', 'Q3']
df_quantils = df_quantils.reset_index()

In [13]:
# CÀLCUL DE L'INDEX ACUMULAT

df_inflacio['index_acumulat'] = 100  # Índex base 100 pel 2013
for i in range(1, len(df_inflacio)):
    df_inflacio.loc[i, 'index_acumulat'] = df_inflacio.loc[i-1, 'index_acumulat'] * (1 + df_inflacio.loc[i, 'variacio_IPC'] / 100)
df_inflacio

/var/folders/vp/zmdnd3gj74q125lmqlsf_b000000gn/T/ipykernel_29994/2115476268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100.584' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_inflacio.loc[i, 'index_acumulat'] = df_inflacio.loc[i-1, 'index_acumulat'] * (1 + df_inflacio.loc[i, 'variacio_IPC'] / 100)


,any,variacio_IPC,index_acumulat
0,2013,0.0,100.000000
1,2014,-1.0,99.000000
2,2015,0.0,99.000000
3,2016,1.6,100.584000
4,2017,1.1,101.690424
5,2018,1.2,102.910709
6,2019,0.8,103.733995
7,2020,-0.5,103.215325
8,2021,6.5,109.924321
9,2022,5.7,116.190007


In [14]:
# Es realitza un merge per obtenir un df combinat del 'df_alquiler' i 'df_inflacio'
df_lloguer_inflacio = pd.merge(df_quantils, df_inflacio, on='any')

# Es calculen els valors ajustats per les variacions de l'IPC
df_lloguer_inflacio['Q1_ajustat'] = df_lloguer_inflacio['Q1'] * 100 / df_lloguer_inflacio['index_acumulat']
df_lloguer_inflacio['mediana/Q2_ajustat'] = df_lloguer_inflacio['mediana/Q2'] * 100 / df_lloguer_inflacio['index_acumulat']
df_lloguer_inflacio['Q3_ajustat'] = df_lloguer_inflacio['Q3'] * 100 / df_lloguer_inflacio['index_acumulat']

df_lloguer_inflacio[['any', 'Q1_ajustat', 'mediana/Q2_ajustat', 'Q3_ajustat']]


,any,Q1_ajustat,mediana/Q2_ajustat,Q3_ajustat
0,2013,587.460000,637.390000,722.545000
1,2014,592.722222,647.282828,739.969697
2,2015,624.000000,707.636364,791.101010
3,2016,658.002267,766.931122,854.285970
4,2017,722.683583,828.327749,918.478814
5,2018,761.072397,878.946426,956.144418
6,2019,789.422987,911.494831,998.954106
7,2020,797.459100,900.941795,988.346452
8,2021,713.449939,808.137810,876.243758
9,2022,751.458771,860.796917,932.644748


In [15]:
lloguers_ajustats = pd.concat([df_lloguer_inflacio['Q1_ajustat'], df_lloguer_inflacio['mediana/Q2_ajustat'], df_lloguer_inflacio['Q3_ajustat']])

quantile_33 = lloguers_ajustats.quantile(0.3333)
quantile_66 = lloguers_ajustats.quantile(0.6667)

In [16]:
# CÀLCUL DELS LLOGUERS AMB LES VARIACIONS DE L'IPC DESCOMPTADES

df_lloguer_districte = df_lloguer[df_lloguer['districtes municipals'] != 'Barcelona']

df_inflacio['acumulacio'] = (1 + df_inflacio['variacio_IPC'] / 100).cumprod()

# Es normalitza de manera que l'any base 2013 té com a valor 1
df_inflacio['acumulacio'] /= df_inflacio.loc[df_inflacio['any'] == 2013, 'acumulacio'].values[0]


In [17]:
# S'uneixen els df en la columna 'any'
df_lloguer_mapa = pd.merge(df_lloguer_districte, df_inflacio[['any', 'acumulacio']], on='any', how='left')

# Es calcula el 'lloguer_ajustat'
df_lloguer_mapa['lloguer_ajustat'] = df_lloguer_mapa['lloguer'] / df_lloguer_mapa['acumulacio']

columnes = ['codi', 'districtes municipals', 'any', 'lloguer', 'lloguer_ajustat']
df_lloguer_mapa = df_lloguer_mapa[columnes]


In [18]:
df_lloguer_mapa.head(5)

,codi,districtes municipals,any,lloguer,lloguer_ajustat
0,1.0,Ciutat Vella,2023,1079.01,900.737048
1,2.0,Eixample,2023,1274.96,1064.312385
2,3.0,Sants-Montjuïc,2023,1032.20,861.660949
3,4.0,Les Corts,2023,1294.20,1080.373571
4,5.0,Sarrià-Sant Gervasi,2023,1550.79,1294.570028


#### COMPRA_DISTRICTES

In [19]:
# CÀLCUL DELS QUANTILS

df_quantils = df_compra.groupby('any')['lloguer'].quantile([0.25, 0.5, 0.75]).unstack()
df_quantils.columns = ['Q1', 'mediana/Q2', 'Q3']
df_quantils = df_quantils.reset_index()

In [20]:
# CÀLCUL DE L'INDEX ACUMULAT

df_inflacio['index_acumulat'] = 100  # Índex base 100 pel 2013
for i in range(1, len(df_inflacio)):
    df_inflacio.loc[i, 'index_acumulat'] = df_inflacio.loc[i-1, 'index_acumulat'] * (1 + df_inflacio.loc[i, 'variacio_IPC'] / 100)
df_inflacio

/var/folders/vp/zmdnd3gj74q125lmqlsf_b000000gn/T/ipykernel_29994/2115476268.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100.584' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_inflacio.loc[i, 'index_acumulat'] = df_inflacio.loc[i-1, 'index_acumulat'] * (1 + df_inflacio.loc[i, 'variacio_IPC'] / 100)


,any,variacio_IPC,index_acumulat,acumulacio
0,2013,0.0,100.000000,1.000000
1,2014,-1.0,99.000000,0.990000
2,2015,0.0,99.000000,0.990000
3,2016,1.6,100.584000,1.005840
4,2017,1.1,101.690424,1.016904
5,2018,1.2,102.910709,1.029107
6,2019,0.8,103.733995,1.037340
7,2020,-0.5,103.215325,1.032153
8,2021,6.5,109.924321,1.099243
9,2022,5.7,116.190007,1.161900


In [21]:
# Es realitza un merge per obtenir un df combinat del 'df_alquiler' i 'df_inflacio'
df_compra_inflacio = pd.merge(df_quantils, df_inflacio, on='any')

# Es calculen els valors ajustats per les variacions de l'IPC
df_compra_inflacio['Q1_ajustat'] = df_compra_inflacio['Q1'] * 100 / df_compra_inflacio['index_acumulat']
df_compra_inflacio['mediana/Q2_ajustat'] = df_compra_inflacio['mediana/Q2'] * 100 / df_compra_inflacio['index_acumulat']
df_compra_inflacio['Q3_ajustat'] = df_compra_inflacio['Q3'] * 100 / df_compra_inflacio['index_acumulat']

df_compra_inflacio[['any', 'Q1_ajustat', 'mediana/Q2_ajustat', 'Q3_ajustat']]


,any,Q1_ajustat,mediana/Q2_ajustat,Q3_ajustat
0,2013,2280.437500,2921.500000,3282.687500
1,2014,2315.656566,3065.530303,3562.689394
2,2015,2340.277778,3313.888889,3792.803030
3,2016,2461.934801,3454.451006,4168.033683
4,2017,2908.644082,3977.267319,4862.859555
5,2018,3136.821259,4149.835365,4731.528957
6,2019,3217.062553,3913.133789,4482.679483
7,2020,3163.229401,3790.740385,4413.709892
8,2021,2865.664736,3531.179423,4009.349309
9,2022,2777.831397,3335.054446,4030.144341


In [22]:
compra_ajustats = pd.concat([df_compra_inflacio['Q1_ajustat'], df_compra_inflacio['mediana/Q2_ajustat'], df_compra_inflacio['Q3_ajustat']])

quantile_33_compra = compra_ajustats.quantile(0.3333)
quantile_66_compra = compra_ajustats.quantile(0.6667)

In [23]:
# CÀLCUL DELS LLOGUERS AMB LES VARIACIONS DE L'IPC DESCOMPTADES

df_inflacio['acumulacio'] = (1 + df_inflacio['variacio_IPC'] / 100).cumprod()

# Es normalitza de manera que l'any base 2013 té com a valor 1
df_inflacio['acumulacio'] /= df_inflacio.loc[df_inflacio['any'] == 2013, 'acumulacio'].values[0]

In [24]:
# S'uneixen els df en la columna 'any'
df_compra_mapa = pd.merge(df_compra, df_inflacio[['any', 'acumulacio']], on='any', how='left')

# Es calcula el 'lloguer_ajustat'
df_compra_mapa['lloguer_ajustat'] = df_compra_mapa['lloguer'] / df_compra_mapa['acumulacio']

columnes = ['codi', 'districtes municipals', 'any', 'lloguer', 'lloguer_ajustat']
df_compra_mapa = df_compra_mapa[columnes]


In [25]:
# Càrrega de les dades dels districtes de l'arxiu GeoJSON
gdf_districtes = gpd.read_file('0301100100_UNITATS_ADM_POLIGONS.json')
gdf_districtes = gdf_districtes[gdf_districtes['TIPUS_UA'] == 'DISTRICTE']

gdf_compra_districtes = gdf_districtes.merge(df_compra_mapa, how='left', left_on=['FID'], right_on=['codi'])

#### BARRIS

In [26]:
df_lloguer_barri.head(5)

,codi,districtes municipals,any,lloguer
0,1,el Raval,2023,959.5
1,2,el Barri Gòtic,2023,1265.38
2,3,la Barceloneta,2023,991.03
3,4,"Sant Pere, Santa Caterina i la Ribera",2023,1138.02
4,5,el Fort Pienc,2023,1179.55


In [27]:
# CÀLCUL DELS LLOGUERS AMB LES VARIACIONS DE L'IPC DESCOMPTADES

df_inflacio['acumulacio'] = (1 + df_inflacio['variacio_IPC'] / 100).cumprod()

# Es normalitza de manera que l'any base 2013 té com a valor 1
df_inflacio['acumulacio'] /= df_inflacio.loc[df_inflacio['any'] == 2013, 'acumulacio'].values[0]

In [28]:
df_lloguer_mapa_barri = pd.merge(df_lloguer_barri, df_inflacio[['any', 'acumulacio']], on='any', how='left')

def calcular_lloguer_ajustat(row):
    try:
        return row['lloguer'] / row['acumulacio']
    except Exception:
        return 'nd'

df_lloguer_mapa_barri['lloguer_ajustat'] = df_lloguer_mapa_barri.apply(calcular_lloguer_ajustat, axis=1)


In [29]:
# Càrrega de les dades dels districtes de l'arxiu GeoJSON
gdf_barris = gpd.read_file('barris.geojson')
anys = pd.DataFrame({'any': range(2013, 2024)})
gdf_barris = gdf_barris.loc[gdf_barris.index.repeat(len(anys))].reset_index(drop=True)
gdf_barris['any'] = pd.concat([anys]*len(gdf_barris['BARRI'].unique()), ignore_index=True)

In [30]:
# Eliminar 0 a l'esquerra de la columna 'BARRI' i convertir a int
gdf_barris['BARRI'] = gdf_barris['BARRI'].str.lstrip('0').astype(int)
gdf_lloguer_barris = gdf_barris.merge(df_lloguer_mapa_barri, how='left', left_on=['BARRI'], right_on=['codi'])

#### COMPRA_BARRIS

In [31]:
# CÀLCUL DELS LLOGUERS AMB LES VARIACIONS DE L'IPC DESCOMPTADES

df_inflacio['acumulacio'] = (1 + df_inflacio['variacio_IPC'] / 100).cumprod()

# Es normalitza de manera que l'any base 2013 té com a valor 1
df_inflacio['acumulacio'] /= df_inflacio.loc[df_inflacio['any'] == 2013, 'acumulacio'].values[0]

In [32]:
df_compra_mapa_barri = pd.merge(df_compra_barri, df_inflacio[['any', 'acumulacio']], on='any', how='left')

def calcular_lloguer_ajustat(row):
    try:
        return row['lloguer'] / row['acumulacio']
    except Exception:
        return 'nd'

df_compra_mapa_barri['lloguer_ajustat'] = df_compra_mapa_barri.apply(calcular_lloguer_ajustat, axis=1)

In [33]:
df_compra_mapa_barri.head(6)

,codi,districtes municipals,any,lloguer,acumulacio,lloguer_ajustat
0,1,el Raval,2023,3386,1.197919,2826.568468
1,2,el Barri Gòtic,2023,4923,1.197919,4109.626867
2,3,la Barceloneta,2023,5154,1.197919,4302.461278
3,4,"Sant Pere, Santa Caterina i la Ribera",2023,4909,1.197919,4097.939933
4,5,el Fort Pienc,2023,4640,1.197919,3873.383843
5,6,la Sagrada Família,2023,4529,1.197919,3780.723152


In [34]:
gdf_compra_barris = gdf_barris.merge(df_compra_mapa_barri, how='left', left_on=['BARRI'], right_on=['codi'])

## VISUALITZACIÓ

In [35]:
gdf_districtes = gpd.read_file('0301100100_UNITATS_ADM_POLIGONS.json')
gdf_districtes = gdf_districtes[gdf_districtes['TIPUS_UA'] == 'DISTRICTE']
gdf_lloguer_districtes = gdf_districtes.merge(df_lloguer_mapa, how='left', left_on=['FID'], right_on=['codi'])

anys = df_padro_obra_nova_districtes['any'].unique()

def categoria_lloguer(row, quantile_33, quantile_66):
    if row['lloguer_ajustat'] == 'nd':
        return 'nd'
    elif row['lloguer_ajustat'] < quantile_33:
        return 'rang baix'
    elif quantile_33 <= row['lloguer_ajustat'] < quantile_66:
        return 'rang mitjà'
    elif row['lloguer_ajustat'] >= quantile_66:
        return 'rang alt'


gdf_lloguer_districtes['categoria'] = gdf_lloguer_districtes.apply(lambda row: categoria_lloguer(row, quantile_33, quantile_66), axis=1)
gdf_lloguer_barris['categoria'] = gdf_lloguer_barris.apply(lambda row: categoria_lloguer(row, quantile_33, quantile_66), axis=1)


gdf_compra_districtes['categoria'] = None
for index, row in gdf_compra_districtes.iterrows():
    if row['lloguer_ajustat'] == 'nd':
        gdf_compra_districtes.at[index, 'categoria'] = 'nd'
    elif row['lloguer_ajustat'] < quantile_33_compra:
        gdf_compra_districtes.at[index, 'categoria'] = 'rang baix'
    elif quantile_33_compra <= row['lloguer_ajustat'] < quantile_66_compra:
        gdf_compra_districtes.at[index, 'categoria'] = 'rang mitjà'
    elif row['lloguer_ajustat'] >= quantile_66_compra:
        gdf_compra_districtes.at[index, 'categoria'] = 'rang alt'


gdf_compra_barris['categoria'] = None
for index, row in gdf_compra_barris.iterrows():
    if row['lloguer_ajustat'] == 'nd':
        gdf_compra_barris.at[index, 'categoria'] = 'nd'
    elif row['lloguer_ajustat'] < quantile_33_compra:
        gdf_compra_barris.at[index, 'categoria'] = 'rang baix'
    elif quantile_33_compra <= row['lloguer_ajustat'] < quantile_66_compra:
        gdf_compra_barris.at[index, 'categoria'] = 'rang mitjà'
    elif row['lloguer_ajustat'] >= quantile_66_compra:
        gdf_compra_barris.at[index, 'categoria'] = 'rang alt'

In [36]:
def mapa_districtes (any, compra_lloguer, habitatges_turistics):
    if compra_lloguer == 'Lloguer':
        gdf_districtes_mapa = gdf_lloguer_districtes[gdf_lloguer_districtes['any'] == any]
    else:
        gdf_districtes_mapa = gdf_compra_districtes[gdf_compra_districtes['any'] == any]
    
    colors = {'rang baix': 'green', 'rang mitjà': 'yellow', 'rang alt': 'red'}
    mapa = folium.Map(location=[41.3851, 2.1734], zoom_start=11)

    folium.GeoJson(
        gdf_districtes_mapa,
        style_function=lambda feature: {
            'fillColor': colors.get(feature['properties'].get('categoria', 'grey'), 'grey'),
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.5
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['NOM'],
            aliases=['Barri:'],
            localize=True,
        ),
        popup=folium.GeoJsonPopup(
            fields=['NOM', 'lloguer'],
            aliases=['Barri:', 'Lloguer mitjà:'],
            localize=True
        ),
        highlight_function=lambda x: {'weight': 3, 'color': 'blue'}
    ).add_to(mapa)


    if habitatges_turistics:
        fol = folium.FeatureGroup(name='Habitatges turístics')
        for id, fila in df_habitatges_turistic.iterrows():
                folium.CircleMarker(
                location=[fila['LATITUD_Y'], fila['LONGITUD_X']],
                radius=0.05,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6,
                popup=fila.get('CARRER', 'No hi ha adressa') 
            ).add_to(fol)
        fol.add_to(mapa)
        folium.LayerControl().add_to(mapa)

    return mapa



def mapa_barris (any, compra_lloguer, habitatges_turistics):
    if compra_lloguer == 'Lloguer':
        gdf_barris_mapa = gdf_lloguer_barris[gdf_lloguer_barris['any_y'] == any]
    else:
        gdf_barris_mapa = gdf_compra_barris[gdf_compra_barris['any_y'] == any]
    
    colors = {'rang baix': 'green', 'rang mitjà': 'yellow', 'rang alt': 'red'}
    mapa = folium.Map(location=[41.3851, 2.1734], zoom_start=11)

    folium.GeoJson(
        gdf_barris_mapa,
        style_function=lambda feature: {
            'fillColor': colors.get(feature['properties'].get('categoria', 'grey'), 'grey'),
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.5
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['NOM'],
            aliases=['Barri:'],
            localize=True,
        ),
        popup=folium.GeoJsonPopup(
            fields=['NOM', 'lloguer'],
            aliases=['Barri:', 'Lloguer mitjà:'],
            localize=True
        ),
        highlight_function=lambda x: {'weight': 2, 'color': 'blue'}
    ).add_to(mapa)

    if habitatges_turistics:
        fol = folium.FeatureGroup(name='Habitatges turístics')
        for id, fila in df_habitatges_turistic.iterrows():
            folium.CircleMarker(
                location=[fila['LATITUD_Y'], fila['LONGITUD_X']],
                radius=0.05,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6,
                popup=fila.get('CARRER', 'No hi ha adressa') 
            ).add_to(fol)
        fol.add_to(mapa)
        folium.LayerControl().add_to(mapa)
        
    return mapa

In [37]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Sector de l'habitatge a Barcelona"),
    dcc.RadioItems(
        id='opcio_barris_districtes',
        options=[
            {'label': 'Mapa de Barris', 'value': 'barris'},
            {'label': 'Mapa de Districtes', 'value': 'districtes'}
        ],
        value='barris',
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    html.P("Selecciona entre les opcions compra i lloguer:"),
    dcc.RadioItems(
        id='opcio_compra_lloguer',
        options=[
            {'label': 'Compra', 'value': 'Compra'},
            {'label': 'Lloguer', 'value': 'Lloguer'}
        ],
        value='Lloguer',
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    html.P("Prémer l'àrea del districte o barri per a més informació"),
    dcc.Dropdown(
        id='opcio_any',
        options=[{'label': str(year), 'value': year} for year in range(2013, 2024)],
        value=2023
    ),
    dcc.Checklist(
        id='opcio_habitatges_turistics',
        options=[
            {'label': 'Habitatges turístics', 'value': 'habitatges'}
        ],
        value=[]
    ),
    html.Div([
        html.Iframe(id='map', width='100%', height='500')
        ]),
    html.Br(),
    html.H1("Lloguer i renda per districtes del 2015 al 2021"),
    dcc.Checklist(
        id='opcio_districte',
        options=[{'label': districte, 'value': districte} for districte in df_renda_lloguer['Districtes municipals'].unique()],
        value=['Barcelona'],
        labelStyle={'display': 'block'}
    ),
    dcc.Graph(id='grafic'),
    html.Br(),
    html.H1("Evolució de l'obra nova i el padró a Barcelona"),
    dcc.Dropdown(
        id='opcio_grafic_barres',
        options=[{'label': str(any), 'value': any} for any in anys],
        value=anys[0]
    ),
    dcc.Graph(id='grafic_barres')
    ])



@app.callback(
    Output('map', 'srcDoc'),
    [Input('opcio_barris_districtes', 'value'),
     Input('opcio_any', 'value'),
     Input('opcio_compra_lloguer', 'value'),
     Input('opcio_habitatges_turistics', 'value')]
)



def creacio_mapa (opcio_barris_districte, opcio_any, opcio_lloguer_renda, opcio_habitatges_turistics):
    hab_tur = 'habitatges' in opcio_habitatges_turistics
    if opcio_barris_districte == 'barris':
        barrio_map = mapa_barris(opcio_any, opcio_lloguer_renda, hab_tur)
        barrio_map.save('mapa_barrios_barcelona.html')
        return open('mapa_barrios_barcelona.html', 'r').read()
    else:
        distrito_map = mapa_districtes(opcio_any, opcio_lloguer_renda, hab_tur)
        distrito_map.save('mapa_distritos_barcelona.html')
        return open('mapa_distritos_barcelona.html', 'r').read()

@app.callback(
    Output('grafic', 'figure'),
    [Input('opcio_districte', 'value')]
)


def grafic_despres_seleccio(districtes_seleccionats):
    df_compra_seleccio = df_renda_lloguer[df_renda_lloguer['Districtes municipals'].isin(districtes_seleccionats)]
    
    linea_lloguer = px.line(df_compra_seleccio, x='Any', y='mitjana_preu_lloguer', color='Districtes municipals',
                  labels={'mitjana_preu_lloguer': 'Mitjana Preu Lloguer'}, title="Mitjana Preu Lloguer")
    
    linea_renda = px.line(df_compra_seleccio, x='Any', y='renda_mitjana_mensual', color='Districtes municipals',
                   labels={'renda_mitjana_mensual': 'Renda Mitjana Mensual'}, title="Renda Mitjana Mensual")
    
    for tipus_linea in linea_renda.data:
        tipus_linea.update(line=dict(dash='dash'))
        linea_lloguer.add_trace(tipus_linea)
    
    linea_lloguer.update_layout(
        xaxis_title="Any",
        yaxis_title="Cantidad",
        legend_title="Districtes municipals",
        template="plotly_white"
    )
    
    linea_lloguer.update_traces(mode='lines+markers')
    linea_lloguer.update_yaxes(title_text="Mitjana preu lloguer / Renda mitjana mensual")

    return linea_lloguer




@app.callback(
    Output('grafic_barres', 'figure'),
    [Input('opcio_grafic_barres', 'value')]
)
def grafic_barres (any_seleccionat):
    df_any = df_padro_obra_nova_districtes[df_padro_obra_nova_districtes['any'] == any_seleccionat]
    fig = go.Figure(data=[
        go.Bar(name='Obra Nova', x=df_any['districtes municipals'], y=df_any['obra_nova']),
        go.Bar(name='Evolució del Padró', x=df_any['districtes municipals'], y=df_any['evolucio_padro'], marker_pattern_shape="/")
    ])
    fig.update_layout(
        title=f"Evolució de l'obra nova i el padró a Barcelona - {any_seleccionat}",
        xaxis_title="Districtes",
        yaxis_title="Valor",
        barmode='group',
        xaxis_tickangle=-45,
        yaxis=dict(range=[-25000, 25000])
    )
    return fig



if __name__ == '__main__':
    app.run_server(debug=True)

